In [ ]:
import matplotlib.pyplot as plt
import numpy as np, seaborn as sns, pandas as pd

In [ ]:
sales_yoy_gwth_pct = 0.2
sales_yoy_gwth_pct_normal_dist = np.random.normal(loc=sales_yoy_gwth_pct, scale=0.02, size=10000)
fig, ax = plt.subplots(figsize=(15,5))
plt.title("Normal distribution for the Sales YoY Growth Rate(%)")
ax1 = sns.histplot(sales_yoy_gwth_pct_normal_dist, label='normal dist', ax=ax, kde=True, stat='density', linewidth=0)
plt.legend()
plt.show()

In [ ]:
years = [2021,2022,2023,2024,2025,2026]
df = pd.DataFrame(index=years, columns=['sales', 'int_exp', 'var_nwc'])
df.iat[0,0] = 250
df.iat[0,1] = -7.5
df.iat[0,2] = -3

for i in range(1, len(years)):
    df.loc[df.index[i],'sales'] = df.loc[df.index[i - 1],'sales'] * (1 + 0.2)
    df.loc[df.index[i],'int_exp'] = df.loc[df.index[i - 1],'int_exp'] * (1 + 0.2)
    df.loc[df.index[i],'var_nwc'] = df.loc[df.index[i - 1],'var_nwc'] * (1 + 0.2)

df['cogs'] = -df['sales'] * 0.63
df['sga'] = -df['sales'] * 0.15
df['dep_amort'] = -df['sales'] * 0.015
df['profit_bef_tax'] = (df['sales'] + df['cogs'] + df['sga'] + df['dep_amort'] + df['int_exp'])
df['tax'] = -df['profit_bef_tax'] * 0.33
df['net_income'] = df['profit_bef_tax'] + df['tax']
df['capex'] = df['dep_amort'] * 1
df['fcf'] = df['net_income'] - df['int_exp'] - df['dep_amort'] + df['var_nwc'] + df['capex']

wacc = 0.09
perp_gwth_rate = 0.015
dfc_factor = [1/(1 + wacc) ** x for x in range(1, len(years))]
term_value = (df['fcf'].iloc[-1] * (1 + perp_gwth_rate) / (wacc - perp_gwth_rate)) * dfc_factor[-1]

npv = df['fcf'][1:] * dfc_factor
comp_value = npv.sum() + term_value
comp_value

In [ ]:
class DCFMonteCarlo: 
    def __init__(self, **kwargs):
        valid_arg = ["num_years","initial_sales","sales_gwt","sales_gwt_stddev","int_exp_rate", "int_exp_rate_stddev",
                    "var_nwc_gwt","var_nwc_gwt_stddev","cogs_rate", "cogs_rate_stddev", 
                    "sga_rate", "sga_rate_stddev","dep_amort_driver","dep_amort_driver_stddev","tax_rate",
                    "capex_da_ratio","capex_da_ratio_stddev","perp_gwth_rate","wacc","num_simulations"]
        for key in valid_arg:
            setattr(self, key, kwargs.get(key))
    
    def profit_bef_tax(self, sales, cogs, sga, dep_amort, int_exp):
        profit_before_tax = sales + cogs + sga + dep_amort + int_exp
        return profit_before_tax
    
    def fcf(self, net_income, int_exp, dep_amort, var_nwc, capex):
        fcf = net_income - int_exp - dep_amort + var_nwc + capex
        return fcf
    
    def normal_dist(self, metric, metric_stddev):
        return np.random.normal(loc=metric, scale=metric_stddev, size=(self.num_years, self.num_simulations))

    def disc_fcf_coeff(self):
        return [1 / (1 + self.wacc) ** n for n in range(1, self.num_years + 1)]

    def dsctd_fcf(self, fcf):
        dsc = sum((fcf.transpose() * self.disc_fcf_coeff()).transpose())
        return dsc
    
    def terminal_value(self, fcf):
        tv = (fcf[-1] * ((1 + self.perp_gwth_rate) / (self.wacc - self.perp_gwth_rate))) * self.disc_fcf_coeff()[-1]
        return tv
    
    def sales(self, sales_gwt):
        sales_gwt = 1 + sales_gwt
        for x in range(1, self.num_years):
            sales_gwt[x] = sales_gwt[x] * sales_gwt[x - 1]
        sales  = self.initial_sales * sales_gwt
        return sales

    def estimate(self):
        stored_values = []
        sales_gwt = self.normal_dist(self.sales_gwt, self.sales_gwt_stddev)
        int_exp_rate = self.normal_dist(self.int_exp_rate, self.int_exp_rate_stddev)
        var_nwc_gwt = self.normal_dist(self.var_nwc_gwt, self.var_nwc_gwt_stddev)
        cogs_rate = self.normal_dist(self.cogs_rate, self.cogs_rate_stddev)
        sga_rate = self.normal_dist(self.sga_rate, self.sga_rate_stddev)
        dep_amort_driver = self.normal_dist(self.dep_amort_driver, self.dep_amort_driver_stddev)
        capex_da_ratio = self.normal_dist(self.capex_da_ratio, self.capex_da_ratio_stddev)
        sales = self.sales(sales_gwt)
        cogs = -sales * cogs_rate
        sga = -sales * sga_rate
        dep_amort = -sales * dep_amort_driver
        int_exp = -sales * int_exp_rate
        profit_bef_tax = self.profit_bef_tax(sales, cogs, sga, dep_amort, int_exp)
        tax = -profit_bef_tax * 0.33 #tax_rate
        net_income = profit_bef_tax + tax
        capex = dep_amort * capex_da_ratio
        var_nwc = capex * var_nwc_gwt
        fcf = self.fcf(net_income, int_exp, dep_amort, var_nwc, capex)
        disc_fcf_coeff = self.disc_fcf_coeff()
        dsctd_fcf = self.dsctd_fcf(fcf)
        terminal_value = self.terminal_value(fcf)
        final_estimation = terminal_value + dsctd_fcf
        stored_values.append(final_estimation)
        return stored_values


In [ ]:
params= {
"num_years":5,
"initial_sales" : 250,
"sales_gwt":0.2,
"sales_gwt_stddev":0.02,
"int_exp_rate":0.03,
"int_exp_rate_stddev":0.005,
'var_nwc_gwt':0.8,
"var_nwc_gwt_stddev":0.02,
'cogs_rate':0.63,
"cogs_rate_stddev":0.015,
'sga_rate':0.15,
"sga_rate_stddev":0.01,
"dep_amort_driver":0.015,
"dep_amort_driver_stddev":0.001,
'capex_da_ratio':1,
"capex_da_ratio_stddev":0.01,
'tax_rate':0.33,
"perp_gwth_rate":0.015,
"wacc":0.09,
"num_simulations":10000
}

company_value = DCFMonteCarlo(**params)
value_range = company_value.estimate()
plt.figure(figsize=(13,5))
plt.title("Company valuation price range | Monte Carlo simulation with 10,000 runs")
plt.hist(value_range,  color="orange", bins=50, )
plt.show()

In [ ]:
df = pd.DataFrame({'company_prices':list(value_range[0])})
df.describe()

In [ ]:
low_price = df.mean() - df.std()
high_price = df.mean() + df.std()
cases_num = df.loc[(df.company_prices >= int(low_price)) & (df.company_prices <= int(high_price))].count() / df.count()
print("Taget price is at ${3}M, low price is ${0}M and its high price is ${1}M in {2}% of \
    the cases".format(low_price.values[0].astype(int),\
        high_price.values[0].astype(int), int(cases_num), int(df.mean().values[0])))
